# batch划分训练

内容来源：https://www.cnblogs.com/gengyi/p/9906614.html

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets('./data/mnist/', one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [2]:
# 查看各样本的子集数量：
# 训练集train样本数
print("train data size:", mnist.train.num_examples)
# train data size:55000)

# 验证集validation样本数
print("validation data size:", mnist.validation.num_examples)
# validation data size:5000

# 测试集 test 样本数
print("test data size:", mnist.test.num_examples)
# test data size:10000

train data size: 55000
validation data size: 5000
test data size: 10000


In [3]:
mnist.train.labels[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [4]:
mnist.train.images[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [5]:
BATCH_SIZE = 200
xs, ys = mnist.train.next_batch(BATCH_SIZE)
print("xs shape:", xs.shape)
# xs.shape(200,784)
print("ys shape:", ys.shape)
# ys.shape(200,10)

xs shape: (200, 784)
ys shape: (200, 10)


In [6]:
x = tf.constant([[1, 2], [1, 2]])
y = tf.constant([[1, 1], [1, 2]])
z = tf.add(x, y)
print(z)
# [[2,3],[2,4]]

Tensor("Add:0", shape=(2, 2), dtype=int32)


In [7]:
A = [[1, 3, 4, 5, 6]]
B = [[1, 3, 4, 3, 2]]
with tf.Session() as sess:
    print(sess.run(tf.equal(A, B)))

[[ True  True  True False False]]


In [8]:
import os
os.path.join('/hello/', 'good/boy/', 'doiido')
# '/hello/good/boy/doiido'

'/hello/good/boy/doiido'

# 前向传播过程神经网络代码

解释:

规定网格输入节点为 （28*28）784 个，将每张图片的像素值转变为一维向量，
隐藏层节点 500 个，因此由输入层到隐藏层的参数 w1 形状为 [784,500]；
输出节点 10 个，因此由隐藏层到输出层的参数 w2 形状为 [500,10]；这里的10，代表数字0-9的十分类
参数满足截断正态分布，并使用正则化，将每个参数的正则化损失加到总损失中。

由输入层到隐藏层的偏置 b1 形状为长度为 500的一维数组，由隐藏层到输出层的偏置 b2 形状为长度为 10 的一维数组，初始化值为全 0。前向传播结构第一层为输入 x 与参数 w1 矩阵相乘加上偏置 b1，再经过 relu 函数，得到隐藏层输出 y1。前向传播结构第二层为隐藏层输出 y1 与参数 w2 矩阵相乘加上偏置 b2，得到输出 y。由于输出 y 要经过 softmax 函数，使其符合概率分布，故输出 y 不经过 relu 函数。

In [16]:
import tensorflow as tf

# 28*28 = 784个像素点，784像素点组成了一个一维数组
INPUT_NODE = 784
# 输出10个数，每个数对应的索引号出现的概率，实现了10分类
OUTPUT_NODE = 10
# 定义了隐藏层节点个数
LAYER1_NODE = 500

# 
def get_weight(shape, regularizer):
    # 在训练神经网络时，随机生成参数 w
    w = tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    # 若使用正则化，则将每个变量的正则化损失加到总损失集合 losses 中
    if regularizer != None: tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w

def get_bias(shape):  
    b = tf.Variable(tf.zeros(shape))  
    return b

# 描述从输入到输出的数据流 
def forward(x, regularizer):
    # 第一层、第二层参数是直接输出的。
    # 第一层参数
    w1 = get_weight([INPUT_NODE, LAYER1_NODE], regularizer)
    b1 = get_bias([LAYER1_NODE])
    y1 = tf.nn.relu(tf.matmul(x, w1) + b1)

    # 第二层参数
    w2 = get_weight([LAYER1_NODE, OUTPUT_NODE], regularizer)
    b2 = get_bias([OUTPUT_NODE])
    y = tf.matmul(y1, w2) + b2
    return y

# 反向传播过程神经网络代码

In [23]:
# 定义每轮喂入多少张图片
BATCH_SIZE = 200
# 最开始的学习率
LEARNING_RATE_BASE = 0.1
# 学习率的衰减率
LEARNING_RATE_DECAY = 0.99
# 正则化系数
REGULARIZER = 0.0001
# 训练多少轮
STEPS = 50000
# 滑动平均衰减率
MOVING_AVERAGE_DECAY = 0.99
# 模型保存路径
MODEL_SAVE_PATH="data/model/"
# 模型保存的文件名
MODEL_NAME="mnist_model"

def backward(mnist):
    # x y_ 用placeholder占位
    x = tf.placeholder(tf.float32, [None, INPUT_NODE])
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE])
    # 调用前向传播的程序，计算y值
    y = forward(x, REGULARIZER)
    # 为轮数计数器赋初值，并设定为不可训练
    global_step = tf.Variable(0, trainable=False)

    # 定义包含正则化的损失函数loss
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cem = tf.reduce_mean(ce)
    loss = cem + tf.add_n(tf.get_collection('losses'))

    # 定义指数衰减学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)

    # 定义训练过程
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # 定义滑动平均
    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    ema_op = ema.apply(tf.trainable_variables())
    with tf.control_dependencies([train_step, ema_op]):
        train_op = tf.no_op(name='train')

    # 实例化saver
    saver = tf.train.Saver()

    # 在会话结构中初始化所有变量
    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)

        for i in range(STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)

def run():
    backward(mnist)

In [24]:
run()

After 1 training step(s), loss on training batch is 3.32263.
After 1001 training step(s), loss on training batch is 0.459273.
After 2001 training step(s), loss on training batch is 0.388798.
After 3001 training step(s), loss on training batch is 0.372842.
After 4001 training step(s), loss on training batch is 0.36131.
After 5001 training step(s), loss on training batch is 0.330846.
After 6001 training step(s), loss on training batch is 0.349259.
After 7001 training step(s), loss on training batch is 0.344689.
After 8001 training step(s), loss on training batch is 0.328936.
After 9001 training step(s), loss on training batch is 0.298664.
After 10001 training step(s), loss on training batch is 0.313234.
After 11001 training step(s), loss on training batch is 0.313227.
After 12001 training step(s), loss on training batch is 0.279158.
After 13001 training step(s), loss on training batch is 0.288505.
After 14001 training step(s), loss on training batch is 0.289478.
After 15001 training step

the code above:

01） 定义每轮喂入神经网络的图片数量、初始学习率、学习率衰减率、正则化系数、训练轮数、模型保存路径以及模型保存名称等相关信息。

02） 在反向传播函数中，

03） 读入mnist，

04） 训练数据 x 和标签 y_ 进行placeholder 占位，

05） 调用 mnist_forward.py 文件中 forward() 函数，并设置正则化，

06） 计算训练数据集上的预测结果 y，

07） 给当前计算轮数计数器赋值，并设定为不可训练类型

08） 调用包含所有参数正则化损失的损失函数 loss

09） 设定指数衰减学习率 learning_rate

10） 使用梯度衰减算法对模型优化，降低损失函数，并定义参数的滑动平均

11） with 结构中，实现所有参数初始化，每次喂入 batch_size 组（本代码为 200 组）训练数据和对应标签，循环迭代 steps 轮，并每隔 1000 轮打印出一次损失函数值信息，并将当前会话加保存到指定路径。

12） 通过函数 run() ，加载指定路径下的训练数据集，并调用规定的 backward() 函数训练模型。

# mnist_test

当训练完模型后，给神经网络模型输入测试集验证网络的准确性和泛化性。注意，所用的测试集和训练集是相互独立的。

在 with 结构中，加载指定路径下的 ckpt，若模型存在，则加载出模型到当前对话，在测试数据集上进行准确率验证，并打印出当前轮数下的准确率，若模型不存在，则打印出模型不存在的提示，从而 test()函数完成。 通过主函数 main()，加载指定路径下的测试数据集，并调用规定的 test 函数，进行模型在测试集上的准确率验证。

In [31]:
import time

# 规定程序循环间隔是 5 秒
TEST_INTERVAL_SECS = 5

def test(mnist):
    # with 语句复现计算图
    with tf.Graph().as_default() as g:
        # placeholder 为x y_占位
        x = tf.placeholder(tf.float32, [None, INPUT_NODE])
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE])
        # 前向传播过程计算y的值
        y = forward(x, None)

        # 实例化带滑动平均的saver对象，这样所有参数在会话中被加载时将被赋值为各自的滑动平均值
        ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        ema_restore = ema.variables_to_restore()
        saver = tf.train.Saver(ema_restore)
        
        # 计算准确率
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        accuracy_score_last=0

        while True:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                # 判单是否有模型
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score_current = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
                    print("After %s training step(s), test accuracy = %g" % (global_step, accuracy_score_current))
                    if accuracy_score_current==accuracy_score_last:
                        return
                    accuracy_score_last=accuracy_score_current
                else:
                    print('No checkpoint file found')
                    return
            #time.sleep(TEST_INTERVAL_SECS)
            
test(mnist)

INFO:tensorflow:Restoring parameters from data/model/mnist_model-49001
After 49001 training step(s), test accuracy = 0.0743
INFO:tensorflow:Restoring parameters from data/model/mnist_model-49001
After 49001 training step(s), test accuracy = 0.0743


在断点处暂停训练并在之后继续训练的内容见原文。